# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [4]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    # def __init__(self, csv, path, transform=None):
    def __init__(self, csv, path, transform=None, strong_transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform
        self.strong_transform = strong_transform # 추가

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # if self.transform:
        #     img = self.transform(image=img)['image']
        
        # 클래스별로 다른 증강 적용
        if self.strong_transform and target in [1, 13, 14]:
            img = self.strong_transform(image=img)['image']
        elif self.transform:
            img = self.transform(image=img)['image']
            
        return img, target

In [5]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [21]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../data/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
# 이부분 튜닝해서 성능을 더 높일 수 있음
img_size = 128
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 0

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [ ]:
# augmentation을 위한 transform 코드
# trn_transform = A.Compose([
#     # 이미지 크기 조정
#     A.Resize(height=img_size, width=img_size),
#     # images normalization
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
#     ToTensorV2(),
# ])
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=45), # 기울기 증강 관련 추가
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# 부족한 클래스용 강한 transform
strong_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.RandomRotate90(p=0.8),
    A.Rotate(limit=45), # 기울기 증강 관련 추가
    A.HorizontalFlip(p=0.7),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.7),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=45, p=0.7),
    A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [8]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../data/train.csv",
    "../data/train/",
    transform=trn_transform,
    strong_transform=strong_transform # 추가
)
tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [9]:
print(trn_dataset.df)
print(trn_dataset[0][0].shape)
print(trn_dataset[0][1])

[['002f99746285dfdd.jpg' 16]
 ['008ccd231e1fea5d.jpg' 10]
 ['008f5911bfda7695.jpg' 10]
 ...
 ['ff51dd281a8423f1.jpg' 11]
 ['ff8a6a251ce51c95.jpg' 5]
 ['ffc22136f958deb1.jpg' 9]]
torch.Size([3, 128, 128])
16


In [10]:
tst_dataset.df

array([['0008fdb22ddce0ce.jpg', 0],
       ['00091bffdffd83de.jpg', 0],
       ['00396fbc1f6cc21d.jpg', 0],
       ...,
       ['ffc2c91dff8cf2c0.jpg', 0],
       ['ffc4e330a5353a2a.jpg', 0],
       ['ffc71fed753d90c1.jpg', 0]], dtype=object)

In [11]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [12]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [22]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 0.7572: 100%|██████████| 50/50 [00:06<00:00,  7.49it/s]


train_loss: 0.3348
train_acc: 0.8834
train_f1: 0.8682
epoch: 0.0000



Loss: 0.6490: 100%|██████████| 50/50 [00:06<00:00,  7.58it/s]


train_loss: 0.2974
train_acc: 0.8962
train_f1: 0.8843
epoch: 1.0000



Loss: 3.4634: 100%|██████████| 50/50 [00:06<00:00,  7.50it/s]


...

train_loss: 0.1564
train_acc: 0.9452
train_f1: 0.9405
epoch: 27.0000



Loss: 1.4810: 100%|██████████| 50/50 [00:06<00:00,  7.53it/s]


train_loss: 0.1699
train_acc: 0.9459
train_f1: 0.9425
epoch: 28.0000



Loss: 2.5266: 100%|██████████| 50/50 [00:06<00:00,  7.53it/s]


train_loss: 0.2042
train_acc: 0.9452
train_f1: 0.9409
epoch: 29.0000



In [ ]:
# validation 과정 추가 !! 

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [23]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:11<00:00,  8.42it/s]


In [24]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [25]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,10
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,10
4,00901f504008d884.jpg,2


In [26]:
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [27]:
# pred_df.to_csv("pred.csv", index=False)

from datetime import datetime

# 현재 시각을 'YYYYMMDD_HHMMSS' 형식으로 가져오기
now = datetime.now().strftime("%Y%m%d_%H%M%S")

# 파일명 생성
filename = f"../data/rslt/pred_{now}.csv"

# 저장
pred_df.to_csv(filename, index=False)

print(f"결과 저장 완료: {filename}")


결과 저장 완료: ../data/rslt/pred_20251102_201016.csv


In [28]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,10
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,10
4,00901f504008d884.jpg,2
